In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Impor Dependences

In [2]:
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

Load Data

In [3]:
data=pd.read_csv('../input/netflix-shows/netflix_titles.csv')
data.head()

# Exploratory data analysis

In [4]:
data.info()

In [5]:
data.columns

In [6]:
data.shape

In [7]:
data.size

In [8]:
data.dtypes

In [9]:
ProfileReport(data)

## Drop duplicate value

In [10]:
data[data.duplicated()]
# data.drop_duplicates(inplace=True)

There is no duplicate data

## Check Null value

In [11]:
NullSum=data.isnull().sum().sort_values(ascending=False)
NullSum

In [12]:
plt.figure(figsize=(15,10))
sns.heatmap(data.isnull())
plt.show()

In [13]:
data["date_added"] = pd.to_datetime(data['date_added'],)
data['year'] = data['date_added'].dt.year
data['month'] = data['date_added'].dt.month

In [14]:
data.head(2)

In [15]:
data.groupby('year')['type'].value_counts()

In [40]:
all_movie_year=data['year'].value_counts()
# all_movie_year=all_movie_year.reset_index()
# px.bar(data_frame=all_movie_year,x='index',y='year')
# px.bar

In [33]:
all_movie_year['index']

In [41]:
px.bar(x=all_movie_year.keys(),y=all_movie_year.values,color=all_movie_year.keys(),title='Top movie relased year')

## Movie ratings analysis

In [44]:
plt.figure(figsize=(12,10))
sns.set(style="darkgrid")
ax = sns.countplot(x="rating", data=data, palette="Set2", order=data['rating'].value_counts().index[0:15])
ax.set_title('Top rating categories')
plt.show()

## show word cloud

In [49]:
from collections import Counter

genres=list(data['listed_in'])
gen=[]

for i in genres:
    i=list(i.split(','))
    for j in i:
        gen.append(j.replace(' ',""))
comment_words=Counter(gen)

In [56]:
comment_words.items()

In [52]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

text = list(set(gen))
plt.rcParams['figure.figsize'] = (13, 13)


stopwords = set(STOPWORDS)
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(str(comment_words))
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()


## Search a keyword in listed_in columns


In [59]:
data[data['description'].str.contains('way into a')][['listed_in','type']]

In [45]:
data.head()

## For "Zombie Dumb" what is the show_id and who is the director

In [15]:
data[data['title'].isin(['Zombie Dumb'])]

In [16]:
data[data['title'].str.contains('Zombie Dumb')]

## which year the highest number of tv show and movie were relesed 

In [17]:
# data['type'].value_counts()
# data.groupby('release_year')['type'].value_counts


In [18]:
px.bar(data['release_year'].value_counts(),title='Top movie released year')

## How many movie and tv show in this dataset 

In [19]:
data['type'].groupby(by=data['type']).count()

In [54]:
import plotly.graph_objects as go
type_group=data['type'].value_counts().reset_index()
type_group.rename(columns={'index':'Type','type':'Count'},inplace=True)

px.pie(data_frame=type_group,values=type_group['Count'],labels=['Movie','Tv show'],names=['Movie','Tv show'])


### Show all the movies that were released in 2000

In [20]:
movie_2000=data[(data['type']=='Movie') & (data['release_year']==2000)]
movie_2000

### Show all the title of tv show in United States

In [21]:
data[(data['type']=='TV Show') & (data['country']=='United States')][['title','description']]

## Show top 10 director who gave the highest number of tv shows or movie

In [30]:
total_director=data.director.unique()
print('There are {} unique directir  in this dataset'.format(len(total_director)))

In [36]:
top_director=data.director.value_counts().head(10)
top_director

In [40]:
top_director.keys()

In [44]:
px.histogram(top_director,x=top_director.keys(),y=top_director.values,color=top_director,title="Top 10 director")

## How many movies tom cruise was cast

In [49]:
data[data['cast'].str.contains('Tom Cruise')]
# data[data['cast']=='Tom Cruise']

*Need to drop na*

In [56]:
new_cast=data.dropna()
new_cast[new_cast['cast'].str.contains('Tom Cruise')]


## show rating category

In [57]:
data['rating'].uniqueique()

There are 17 differenf rating categort

In [58]:
data['rating'].unique()

## show all the movie where rating is TV-Y7

In [64]:
data[(data['type']=='Movie') & (data['rating']=='TV-Y7')][['title','duration','release_year','country']]

## All Tv Show after 2000

In [66]:
data[(data['type']=='TV Show') & (data['release_year']>2000)]

## What is the maximum duration of movies

In [80]:
data['duration'].dtype

In [81]:
data[['Minutes','Units']]=data['duration'].str.split(' ',expand=True)

In [86]:
time=data['Minutes'].dropna().astype(int)
print('Maximim Minutes ',time.max())
print('Minumam Minutes ',time.min())

In [90]:
time

In [94]:
data[data['Minutes']=='312']['title']

## Maximum number of movie released country

In [97]:
max_movie=data.country.value_counts().head(30)
max_movie

In [103]:
fig=px.funnel(max_movie,color=max_movie.keys())
fig.show()